In [1]:
import pandas as pd
import numpy as np
import re
import os

from itertools import chain
from collections import Counter

In [2]:
from preprocessing.basic_preprocessing import *

In [3]:
import os
import sys

sys.path.append(os.path.dirname(os.path.abspath('.')))

from utils import GCSHelper

gcs_helper = GCSHelper(
            "../keys/gcs_key.json"
        )
df = gcs_helper.read_df_from_gcs(
            "csv/user_detail_temp_total.csv"
        )  # 참고사항: temp_total은 아마 지금은 지워졌을 듯

# 1. 요주의 인물 파악하기

## 1.1 user_info 가져오기
- 이건 우리가 크롤링한 총 사람들

In [4]:
user_info_df = pd.DataFrame(columns=["server", "user_info_url", "user"])

for user_info_path in os.listdir('./data/user_info_full'):
        temp = pd.read_csv(os.path.join('./data/user_info_full', user_info_path))
        user_info_df = pd.concat([user_info_df, temp])

user_info_df.columns = ["server", "user_info_url", "nickname"]

user_info_df = user_info_df.drop_duplicates(subset=["nickname"])

## 1.2 user_detail 가져오기
- 이건 우리가 크롤링한것 모든 합본 (1160000 정도, 그래서 중복 있음)

In [5]:
user_detail_df = pd.DataFrame(columns=['nickname', 'codi-hat', 'codi-hair', 'codi-face', 'codi-top',
                                           'codi-bottom', 'codi-shoes', 'codi-weapon', 'level', 'class',
                                           'popularity', 'total_ranking', 'world_ranking', 'class_world_ranking',
                                           'class_total_ranking', 'guild', 'last_access', 'mureung', 'theseed',
                                           'union', 'achievement', 'cur_chr', 'past_chr_img_1', 'past_chr_img_2',
                                           'past_chr_img_3', 'past_chr_img_4', 'past_chr_img_5', 'past_chr_img_6',
                                           'past_chr_date_1', 'past_chr_date_2', 'past_chr_date_3',
                                           'past_chr_date_4', 'past_chr_date_5', 'past_chr_date_6'])

user_detail_df = pd.concat([user_detail_df, df])

user_detail_df = user_detail_df.drop_duplicates(subset=["nickname"])

## 1.3 둘이 합치기
- 84,000명이 아니라 839,820명

In [6]:
user_df = pd.merge(left=user_detail_df, right=user_info_df, on="nickname", how="left")

# 기존에 있던 코드

In [7]:
def data_concat(user_info, user_detail):
    user_info_df = pd.DataFrame(columns=["server", "user_info_url", "user"])

    for user_info_path in os.listdir(user_info):
        temp = pd.read_csv(os.path.join(user_info, user_info_path))
        user_info_df = pd.concat([user_info_df, temp])

    user_info_df.columns = ["server", "user_info_url", "nickname"]

    user_detail_df = pd.DataFrame(columns=['nickname', 'codi-hat', 'codi-hair', 'codi-face', 'codi-top',
                                           'codi-bottom', 'codi-shoes', 'codi-weapon', 'level', 'class',
                                           'popularity', 'total_ranking', 'world_ranking', 'class_world_ranking',
                                           'class_total_ranking', 'guild', 'last_access', 'mureung', 'theseed',
                                           'union', 'achievement', 'cur_chr', 'past_chr_img_1', 'past_chr_img_2',
                                           'past_chr_img_3', 'past_chr_img_4', 'past_chr_img_5', 'past_chr_img_6',
                                           'past_chr_date_1', 'past_chr_date_2', 'past_chr_date_3',
                                           'past_chr_date_4', 'past_chr_date_5', 'past_chr_date_6'])

    for user_detail_path in os.listdir(user_detail):
        temp = pd.read_csv(os.path.join(user_detail, user_detail_path))
        user_detail_df = pd.concat([user_detail_df, temp])

    user_detail_df = user_detail_df.drop_duplicates(subset=["nickname"])

    user_df = pd.merge(left=user_detail_df, right=user_info_df, on="nickname", how="left")

    return user_df

In [8]:
len(user_df)

839820

## data1 2 에 대한 내용은 일단 제외할게요

In [6]:
data = pd.merge(data_1, data_2, on = 'nickname')
caution = data[(data['codi-hat_x']!=data['codi-hat_y'] )| (data['codi-hair_x']!=data['codi-hair_y']) | (data['codi-face_x']!=data['codi-face_y']) | (data['codi-top_x']!=data['codi-top_y']) | (data['codi-bottom_x']!=data['codi-bottom_y']) | (data['codi-shoes_x']!=data['codi-shoes_y']) | (data['codi-weapon_x']!=data['codi-weapon_y'])]
caution = caution['nickname'].reset_index(drop=True)

In [7]:
data = data.sort_values(by='total_ranking_x', ascending=False)
data

,nickname,codi-hat_x,codi-hair_x,codi-face_x,codi-top_x,codi-bottom_x,codi-shoes_x,codi-weapon_x,level_x,class_x,...,past_chr_img_5_y,past_chr_img_6_y,past_chr_date_1_y,past_chr_date_2_y,past_chr_date_3_y,past_chr_date_4_y,past_chr_date_5_y,past_chr_date_6_y,server_y,user_info_url_y
30008,테룽이,해바라기 꽃모자,샤이닝 헤어,꼬맹이 얼굴,별빛 연구원,-,[BTS] 사일런트 나이트,해바라기 큰송이,Lv.290(25.23%),카이저,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-11-24,22-08-05,22-08-03,22-05-02,22-04-21,22-02-26,NaN,NaN
30097,화놋이,하이네스 던위치햇,아침등교 헤어,오르카 얼굴,이글아이 던위치로브,트릭스터 던위치팬츠,앱솔랩스 메이지슈즈,제네시스 스태프,Lv.286(16.846%),"아크메이지(불,독)",...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-24,22-12-22,22-12-15,22-12-12,22-12-03,22-11-15,NaN,NaN
30988,꿈릿,묵빛 갓,블링 루야 헤어,멍한 오드아이 얼굴,정결한 기품,-,-,천년의 약속,Lv.282(60.369%),패스파인더,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-24,22-12-21,22-12-20,22-12-16,22-12-12,22-12-11,NaN,NaN
30989,LeeHaeSeong,흐드러진 악몽,폴암의 전사 헤어,-,서린 장미,-,악몽 행진곡,악몽 진주,Lv.282(61.166%),은월,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-22,22-12-15,22-12-14,22-12-10,22-12-02,22-11-30,NaN,NaN
30978,채경이,피어난 악몽,블레이 헤어,루시드 얼굴,서린 장미,-,악몽 행진곡,악몽 진주,Lv.282(60.173%),아크,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-13,22-11-24,22-11-20,22-11-17,22-11-16,22-10-26,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30991,키프로사,바니 스냅백,멜로우 헤어,-,하트비트 두근,-,러블리 체크 구두,매혹의 향기,Lv.282(59.24%),카인,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-19,22-12-16,22-12-10,22-12-09,22-11-24,22-11-06,NaN,NaN
30984,참미인,시련의 무게,설탕소녀 헤어,졸린 얼굴,레이디 라비린스,-,해답의 조각,미궁의 연무,Lv.282(59.111%),아델,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-11-24,22-09-08,22-07-03,22-06-27,22-06-13,22-04-21,NaN,NaN
31015,무선통신,19주년 생일 모자,-,조용한 눈 얼굴,행복한 생일,-,생일날 양말,-,Lv.282(62.277%),아델,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-22,22-12-21,22-12-18,22-12-17,22-12-16,22-12-13,NaN,NaN
30994,으줴,오로라 왕관,블링 루야 헤어,차분한 헤헤 얼굴,순수한 은하,-,-,오로라 왈츠,Lv.282(60.299%),나이트로드,...,https://avatar.maplestory.nexon.com/Character/...,https://avatar.maplestory.nexon.com/Character/...,22-12-24,22-12-23,22-12-13,22-11-22,22-11-19,22-11-12,NaN,NaN


In [8]:
caution #1-200000 (except 60000-70000)

0         초장파위법사
1            피가영
2        Brendis
3             뮤렬
4          윙비트코인
          ...   
65184      yeoni
65185     아델리펭구구
65186         탓도
65187       향일화린
65188        리랏응
Name: nickname, Length: 65189, dtype: object

# 2. EDA 및 이상치 파악

In [8]:
from google.cloud import storage

def download_from_gcs(
    blob_name: str,
    file_name: str,
    key_path: str = "key.json",
    bucket_name: str = "maple_raw_data",
):
    """
    파일을 gcs로 부터 다운로드

    blob_name: gcs에 저장되어있는 파일 경로 및 이름 (gcs 상의 경로) ex) csv/user_info.csv
    file_name: gcs에서 download할 파일 경로 및 이름 (local 상의 경로) ex) data/user_info/user_info.csv
    key_path: 아까 저장한 key file 경로 ex) config/key.json
    bucket_name: gcs 안에 어떤 버킷에서 불러 올 것인지? ex) maple_raw_data
    """
    # client 가져오기
    storage_client = storage.Client.from_service_account_json(key_path)
    # client 내 bucket 가져오기
    bucket = storage_client.get_bucket(bucket_name)
    # bucket 내 깡통 blob 생성
    blob = bucket.blob(blob_name)
    # 해당 blob에 파일 업로드
    blob.download_to_filename(file_name)
    return None

In [9]:
pwd

'/opt/ml/final-project-level3-recsys-01'

In [63]:
# data = download_from_gcs('csv/user_detail_temp_total.csv', './data/user_detail_total_ver1.csv', key_path='./config/final-project-372103-660e1aea91f7.json')
# data_1 = get_int_basic(data_1)
# data_1 = get_int_record(data_1)

## 기본 꼴 보기

In [76]:
# data = pd.read_csv('./data/user_detail_total_ver1.csv') # df 불러오는 코드

In [10]:
data = df

In [11]:
data.columns

Index(['nickname', 'codi-hat', 'codi-hair', 'codi-face', 'codi-top',
       'codi-bottom', 'codi-shoes', 'codi-weapon', 'level', 'class',
       'popularity', 'total_ranking', 'world_ranking', 'class_world_ranking',
       'class_total_ranking', 'guild', 'last_access', 'mureung', 'theseed',
       'union', 'achievement', 'cur_chr', 'past_chr_img_1', 'past_chr_img_2',
       'past_chr_img_3', 'past_chr_img_4', 'past_chr_img_5', 'past_chr_img_6',
       'past_chr_date_1', 'past_chr_date_2', 'past_chr_date_3',
       'past_chr_date_4', 'past_chr_date_5', 'past_chr_date_6'],
      dtype='object')

'nickname', 닉네임 str
'codi-hat', 모자 정보 str
'codi-hair', 헤어 정보 str
'codi-face', 성형 정보 str
'codi-top', 상의 정보 str
'codi-bottom', 하의 정보 str
'codi-shoes', 신발 정보 str
'codi-weapon', 무기 정보 str
'level', 레벨 Lv.238(75.044%) -> float형 exp와 int형 level로 쪼갬 <경험치바는 의미 없을듯>
'class', 직업 str 
'popularity', 인기도 인기도\n0 -> int
'total_ranking', 660,239위 -> 위 없애고 int 
'world_ranking', 77,290위 -> 위 없애고 int -> 없는 애들은 -1, 위 떼고 int
'class_world_ranking', 직업월드랭킹 (월드)2,726위 -> 없는 애들은 -1, 위 떼고 int
'class_total_ranking', 직업전체랭킹 (전체)24,990위 -> 없는 애들은 -1, 위 떼고 int
'guild', 길드 이름 혹은 여부 (없음)	
'last_access', 22/12/22
'mureung', 무릉 기록이 없습니다. or int -> -1 또는 int로 변환
'theseed', 시드 기록이 없습니다. or int  -> -1 또는 int로 변환
'union', 유니온 기록이 없습니다. or int  -> -1 또는 int로 변환
'achievement', 유니온 기록이 없습니다. or int  -> -1 또는 int로 변환
'cur_chr', 현재 캐릭터 사진 https://avatar.maplestory.nexon.com/Character/...
'past_chr_img_1', 과거 사진1 
'past_chr_img_2',
'past_chr_img_3', 
'past_chr_img_4', 
'past_chr_img_5', 
'past_chr_img_6',
'past_chr_date_1', 과거 사진의 날짜 1 (다 빼)
'past_chr_date_2', (다 빼)
'past_chr_date_3',(다 빼)
'past_chr_date_4', (다 빼)
'past_chr_date_5', (다 빼)
'past_chr_date_6'(다 빼)

In [12]:
# int로 나타낼 것을 나타냄
data = get_int_basic(data)
# 기록이 없습니다. 인 것을 int로 나타냄
data = get_int_record(data)

### 기본 전처리 (날짜) 근데 결국 드랍해버림 사실 의미 없음

In [13]:
date_columns = [x for idx, x in enumerate(data.columns) if x.find('date')!=-1]
date_columns
for date in date_columns:
    data[date] = data[date].apply(lambda x : x.replace('/', '-'))
data.drop(date_columns, axis=1, inplace=True)

In [14]:
data['last_access'] = data['last_access'].apply(lambda x : x.replace('/', '-'))

### NULL 값 파악

In [15]:
data.isna().sum()

nickname               0
codi-hat               0
codi-hair              0
codi-face              0
codi-top               0
codi-bottom            0
codi-shoes             0
codi-weapon            0
level                  0
class                  0
popularity             0
total_ranking          0
world_ranking          0
class_world_ranking    0
class_total_ranking    0
guild                  0
last_access            0
mureung                0
theseed                0
union                  0
achievement            0
cur_chr                0
past_chr_img_1         0
past_chr_img_2         0
past_chr_img_3         0
past_chr_img_4         0
past_chr_img_5         0
past_chr_img_6         0
exp                    0
dtype: int64

## data 1 2는 넘어갈게요

In [14]:
data_1['past_chr_date_6'].value_counts()

None        4196
22-11-24    3180
22-09-18    2837
22-09-17    2383
22-04-21    2207
            ... 
18-12-08       1
18-11-06       1
18-09-26       1
18-09-13       1
20-05-03       1
Name: past_chr_date_6, Length: 1321, dtype: int64

In [15]:
sum(data_1['past_chr_date_1'].str.find('None')!=-1) # 1번째에는 null값 X

0

In [16]:
for date in date_columns:
    if sum(data_1[date].str.find('None')!=-1)!=0:
        print(date, data_1[date].value_counts()['None']/len(data_1))

past_chr_date_2 0.0011578947368421054
past_chr_date_3 0.005105263157894737
past_chr_date_4 0.0108
past_chr_date_5 0.016431578947368422
past_chr_date_6 0.02208421052631579


- 막 엄청 많지는 않네 null값 비율이

##### Null값 처리
- None인 애들은 99-12-31으로 바꿔줌

In [17]:
for date in date_columns:
    data_1[date] = data_1[date].apply(lambda x : x.replace('None', '99-12-31'))
    data_1[date] = data_1[date].apply(lambda x : '20' + x)

In [18]:
data_1['past_chr_date_6'].value_counts()

2099-12-31    4196
2022-11-24    3180
2022-09-18    2837
2022-09-17    2383
2022-04-21    2207
              ... 
2018-12-08       1
2018-11-06       1
2018-09-26       1
2018-09-13       1
2020-05-03       1
Name: past_chr_date_6, Length: 1321, dtype: int64

#### 입고 있는 코디 정보

In [15]:
codi_columns = [x for idx, x in enumerate(data.columns) if x.find('codi')!=-1]

In [16]:
for codi in codi_columns:
    print(codi, data[codi].value_counts()['-'])

codi-hat 127462
codi-hair 235483
codi-face 677799
codi-top 47064
codi-bottom 1101133
codi-shoes 328926
codi-weapon 132663


- 116만개 중에 codi-bottom은 110만개가 -인걸 보아 codi-bottom은 없애는게 나을지도 모르겠음
- 성형 정보도 절반 가량은 -인데 어떡하지?

#### 길드/무릉도장/더시드/유니온
- 길드의 경우 Null은 (없음)
- 무릉도장/더시드/유니온의 경우 Null은 -1

In [17]:
# 길드의 Null값 비율
data['guild'].value_counts()['(없음)']/len(data)

0.3644913793103448

In [18]:
# 무릉도장의 null값 비율
data['mureung'].value_counts()[-1]/len(data)

0.36541293103448275

In [19]:
# 더시드의 null값 비율
data['theseed'].value_counts()[-1]/len(data)

0.7091931034482759

In [20]:
# 유니온의 null값 비율 -> 일정 수치 이하는 집계가 안 될 수도 있음 + 부캐는 안 뜸
data['union'].value_counts()[-1]/len(data)

0.7712336206896552

## Codi
- 에러 데이터는 성형 쪽에 많음 (무기에도 하나 있긴 함)
- 전체적으로 전처리 한번 싹 돌릴 필요 있어 보임 (했다는게 아니라 할 일)
    - strip
    - 공백 2개 이상 제거

In [26]:
list(chain(*list(data['codi-hat'].str.split(' ').values)))

['그랜드',
 '마스터',
 '햇',
 '하이네스',
 '원더러햇',
 '달달',
 '머리띠',
 '덜바랜',
 '빛',
 '별빛',
 '티아라',
 '-',
 '하이퍼',
 '버닝',
 '크라운',
 '-',
 '사랑의',
 '메신쥐',
 '모자',
 '하이퍼',
 '버닝',
 '크라운',
 '악역',
 '리본',
 '아폴론',
 '하이퍼',
 '버닝',
 '크라운',
 '하이퍼',
 '버닝',
 '크라운',
 '레인보우',
 '캔디',
 '기억의',
 '갈래',
 '하이네스',
 '던위치햇',
 '[BTS]',
 '이클립스',
 '페더',
 '비숑',
 '모자',
 '녹옥',
 '눈모자',
 '신선',
 '수련',
 '모자',
 '하이퍼',
 '버닝',
 '크라운',
 '메이플',
 '아이돌',
 '모자',
 '스윗',
 '디어',
 '후드',
 '낙원의',
 '장미',
 '하이퍼',
 '버닝',
 '크라운',
 '하이퍼',
 '버닝',
 '크라운',
 '달',
 '그림자',
 '캡',
 '어둠의',
 '기사단모',
 '할로윈',
 '고양이',
 '귀',
 '행복이',
 '쌓였담',
 '달달',
 '머리띠',
 '미혹의',
 '파편',
 '달달',
 '머리띠',
 '하이네스',
 '던위치햇',
 '별빛',
 '티아라',
 '핑크빈',
 '심플캡',
 '별빛',
 '악단',
 '모자',
 '메이플',
 '바캉스',
 '모자',
 '예티',
 '심플캡',
 '하이퍼',
 '버닝',
 '크라운',
 '하이퍼',
 '버닝',
 '크라운',
 '가려진',
 '빛',
 '파도의',
 '춤',
 '크왕',
 '상어',
 '베이비핑크',
 '큐트핀',
 '곰도리',
 '귀',
 '달달',
 '머리띠',
 '-',
 '월묘',
 '모자',
 '소가',
 '됐소',
 '머리띠',
 '샤이닝',
 '인플루언서',
 '하이퍼',
 '버닝',
 '크라운',
 '갈색',
 '항공',
 '모자',
 '별의',
 '전언',
 '번개의',
 '기사단모',
 '-',
 '안개',
 '속

In [28]:
def get_sorted_set(column):
    item_list = list(chain(*list(data[column].str.split(' ').values)))
    item_set = Counter(item_list)
    item_set = sorted(item_set.items(), key=lambda kv:kv[1], reverse=True)
    return item_set

### codi-hat

In [29]:
hat_set = get_sorted_set('codi-hat')

직접 살펴 본 결과 'ㅇ', 'ㅁ', 'ㅍ', '' 등의 이상한 글자가 있음 -> 전처리 필요함 -> 안 필요함 있는 아이템 맞음  
그리고 'New'도 있던데 이건 항상 붙는건지 아님 일정 기간만 붙는건지 확인해 볼 필요 있음

In [30]:
hat_set

[('머리띠', 154812),
 ('달달', 134437),
 ('-', 127462),
 ('모자', 114401),
 ('크라운', 77297),
 ('하이퍼', 68002),
 ('버닝', 67898),
 ('악몽', 24479),
 ('[BTS]', 23048),
 ('리본', 22176),
 ('별빛', 21288),
 ('귀', 20843),
 ('기억의', 20825),
 ('갈래', 20825),
 ('페더', 18816),
 ('기사단모', 18672),
 ('메이플', 16943),
 ('캡', 16659),
 ('이클립스', 16270),
 ('래빗', 15224),
 ('시련의', 15151),
 ('무게', 15151),
 ('티아라', 14663),
 ('하이네스', 14501),
 ('토끼', 14221),
 ('블랙', 14219),
 ('베레모', 13954),
 ('별의', 13853),
 ('밤의', 13497),
 ('뿔', 13430),
 ('화이트', 13051),
 ('후드', 11295),
 ('영원', 10910),
 ('맹세', 10910),
 ('흐드러진', 10721),
 ('그림자', 10288),
 ('머리핀', 9357),
 ('마이', 9293),
 ('퀸', 9293),
 ('빛', 9282),
 ('선글라스', 9280),
 ('전언', 9260),
 ('미드나잇', 8925),
 ('사랑의', 8857),
 ('푸른', 8763),
 ('하보크', 8620),
 ('메신쥐', 8524),
 ('낙원의', 8471),
 ('어둠의', 8457),
 ('심플', 8190),
 ('로얄', 7800),
 ('비니', 7750),
 ('할로윈', 7710),
 ('마녀', 7597),
 ('안개', 7581),
 ('가발', 7459),
 ('신선', 7247),
 ('윙', 7202),
 ('크로스', 6963),
 ('미혹의', 6825),
 ('파편', 6825),
 ('페도라', 6798),
 (

In [23]:
len(hat_set)

1659

In [31]:
data['codi-hat'][data['codi-hat'].str.find('#')!=-1].unique()

array([], dtype=object)

In [32]:
data[data['codi-hat'].str.find('ㅇ')!=-1]['codi-hat']

16400      ㅇ 모자
17403      ㅇ 모자
19232      ㅇ 모자
31522      ㅇ 모자
36133      ㅇ 모자
           ... 
1111497    ㅇ 모자
1120242    ㅇ 모자
1140172    ㅇ 모자
1145808    ㅇ 모자
1151944    ㅇ 모자
Name: codi-hat, Length: 112, dtype: object

In [92]:
data[data['codi-hat'].str.find('ㅍ')!=-1]['codi-hat']

21663      ㅍ 모자
42055      ㅍ 모자
50035      ㅍ 모자
286592     ㅍ 모자
324251     ㅍ 모자
354786     ㅍ 모자
373292     ㅍ 모자
396562     ㅍ 모자
468516     ㅍ 모자
482797     ㅍ 모자
502797     ㅍ 모자
528890     ㅍ 모자
616562     ㅍ 모자
706552     ㅍ 모자
766592     ㅍ 모자
866562     ㅍ 모자
936552     ㅍ 모자
1004402    ㅍ 모자
1011056    ㅍ 모자
1046210    ㅍ 모자
1114065    ㅍ 모자
1129170    ㅍ 모자
1151212    ㅍ 모자
Name: codi-hat, dtype: object

In [93]:
data[data['codi-hat'].str.find('ㅁ')!=-1]['codi-hat']

4578       ㅁ 모자
76475      ㅁ 모자
102139     ㅁ 모자
105491     ㅁ 모자
126471     ㅁ 모자
149865     ㅁ 모자
280165     ㅁ 모자
304574     ㅁ 모자
317991     ㅁ 모자
321397     ㅁ 모자
336277     ㅁ 모자
359874     ㅁ 모자
372206     ㅁ 모자
377057     ㅁ 모자
396920     ㅁ 모자
449402     ㅁ 모자
530341     ㅁ 모자
549832     ㅁ 모자
606024     ㅁ 모자
616920     ㅁ 모자
700333     ㅁ 모자
706534     ㅁ 모자
760165     ㅁ 모자
810341     ㅁ 모자
829832     ㅁ 모자
831841     ㅁ 모자
856024     ㅁ 모자
858722     ㅁ 모자
866920     ㅁ 모자
930333     ㅁ 모자
936534     ㅁ 모자
969260     ㅁ 모자
1012925    ㅁ 모자
1068800    ㅁ 모자
1071246    ㅁ 모자
1083791    ㅁ 모자
1095639    ㅁ 모자
1145467    ㅁ 모자
1145959    ㅁ 모자
1150007    ㅁ 모자
1151646    ㅁ 모자
Name: codi-hat, dtype: object

ㅇ모자 ㅍ모자 ㅁ모자라는게 있음? 그럴 수가 있는거임?  
도메인 전문가 말에 의하면 있다고 합니다 장난하세요?  
음 아무래도 메이플은 장난이긴 함

### codi-hair

In [94]:
hair_set = get_sorted_set('codi-hair')
len(hair_set)

804

hat에 비하면 이상한 데이터는 적은 편, 단 '미', '랑', '진'같이 어딘가에 붙어 있어야 할 것 같은 단어들이 있음  
마찬가지로 ''는 있음  
hat에 비해 다양성이 좀 부족한 것 같음

In [26]:
data['codi-hair'][data['codi-hair'].str.find('#')!=-1].unique()

array([], dtype=object)

### codi-face

In [95]:
face_set = get_sorted_set('codi-face')
len(face_set)

434

에러가 난 듯한 데이터가 좀 있는편  
ex) 남자얼굴12, 여자얼굴6, 여자얼굴21, #20319, #20419, #20619 ... 처럼 #이 붙은 데이터들도 있음  
#붙은 애들은 도메인 지식으로 해결하면 좋을 것 같고 남자얼굴12, 여자얼굴6같은 건 어떻게 해야 할 지 잘 모르겠음  
-> 장난하세요 검색해보니까 성형 이름이 남자얼굴12인거임 진짜 얼굴 이름이 남자얼굴12..그럴수잇음 아무래도  
종류가 상당히 적은 편

In [100]:
data['codi-face'][data['codi-face'].str.find('#')!=-1].unique()

array(['#20010', '#24129', '#24629', '#24529', '#24429', '#21066',
       '#24229', '#24029', '#24329', '#20119', '#24729', '#20719',
       '#20019', '#20072', '#20519', '#20473', '#21668', '#20219',
       '#20419', '#21067', '#20319', '#20619'], dtype=object)

- 이정도는 도메인 지식으로 채워넣어줄 수 있지 않을까?

### codi-top

In [33]:
top_set = get_sorted_set('codi-top')

도메인 지식 없는 내가 보기에는 별 특별한 데이터 없어보임  
종종 한 글자가 나오긴 함  
':'도 있음

In [35]:
data['codi-top'].str.find('#') 

0         -1
1         -1
2         -1
3         -1
4         -1
          ..
1159995   -1
1159996   -1
1159997   -1
1159998   -1
1159999   -1
Name: codi-top, Length: 1160000, dtype: int64

In [36]:
data['codi-top'][data['codi-top'].str.find(':')!=-1].unique()

array(['셀러브리티 티셔츠 : 패숑', '셀러브리티 티셔츠 : 데피', '셀러브리티 티셔츠 : 아무어 '],
      dtype=object)

### codi-bottom

In [102]:
bottom_set = get_sorted_set('codi-bottom')

대부분이 -라서 그런지 종류가 엄청 없고 이상치도 없어보임  
팬츠는 붙여 쓰고 하의는 띄어쓴다는게 발견한 점  
여기에도 ':'가 있음 왜 붙은건지 한번 확인해봐도 될듯

In [28]:
data['codi-bottom'][data['codi-bottom'].str.find('#')!=-1].unique()

array([], dtype=object)

### codi-shoes

In [103]:
shoes_set = get_sorted_set('codi-shoes')

여기도 -가 많음  
여기는 !가 떨어져서 있음 ''도 존재함

In [29]:
data['codi-shoes'][data['codi-shoes'].str.find('#')!=-1].unique()

array([], dtype=object)

### codi-weapons

In [104]:
weapon_set = get_sorted_set('codi-weapon')

-가 많음  
여기에도 New가 존재함  

In [37]:
data['codi-weapon'][data['codi-weapon'].str.find('#')!=-1].unique()

array(['#1482227'], dtype=object)

- 얘 하나만 에러 데이터임

# 레벨 문제는 어쩔 수 없다는 결론이었지

### Numeric 이상치 확인

#### Level

In [32]:
data['level'].describe()

count    1.160000e+06
mean     2.507772e+02
std      1.098594e+01
min      1.000000e+01
25%      2.410000e+02
50%      2.500000e+02
75%      2.570000e+02
max      2.920000e+02
Name: level, dtype: float64

In [43]:
data[data['level']<230][['level', 'total_ranking']]

,level,total_ranking
115726,229,805726
260261,211,240258
494167,226,1298330
499710,10,59598739
501263,217,2555981
510635,217,2484702
523666,181,54621879
618697,10,189459
627482,226,1334042
650325,205,566878


- 레벨이랑 랭킹이 너무 낮은 애들이 긁혀 있음
- info와 detail 크롤링 사이의 시점이 달라서 생기는 문제로 크롤링 자체는 맞게 긁혔음